## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [2]:
import pandas as pd
import math
from pymystem3 import Mystem

In [3]:
bank = pd.read_csv('/datasets/data.csv')
bank.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [4]:
bank.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


В общей информации о нашем датафрейме можно увидеть, что в столбцах `days_employed` и `total_income` есть пропущенные значения, причем их количество одинаковое. Вероятно, пропуски значений в этих столбцах связаны между собой.

Также бросается в глаза отрицательное и дробное значение трудового стажа для не пенсионеров и огромные значения стажа для пенсионеров. Эти значения отличаются возможно из - за того, что они берутся из разных источников. Не пенсионеры предоставляют справки о доходах самостоятельно, в то время когда информацию о пенсионерах запрашивают из пенсионного фонда.
Сами пропуски могут быть связаны со сбоем в передаче данных из пенсионного фонда либо непредоставлением справки у не пенсионеров.

In [5]:
bank[bank['income_type'] == 'пенсионер'].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
18,0,400281.136913,53,среднее,1,вдовец / вдова,2,F,пенсионер,0,56823.777243,на покупку подержанного автомобиля
24,1,338551.952911,57,среднее,1,Не женат / не замужем,4,F,пенсионер,0,290547.235997,операции с коммерческой недвижимостью
25,0,363548.489348,67,среднее,1,женат / замужем,0,M,пенсионер,0,55112.757732,покупка недвижимости
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
30,1,335581.668515,62,среднее,1,женат / замужем,0,F,пенсионер,0,171456.067993,операции с коммерческой недвижимостью
35,0,394021.072184,68,среднее,1,гражданский брак,1,M,пенсионер,0,77805.677436,на проведение свадьбы
50,0,353731.432338,63,среднее,1,женат / замужем,0,F,пенсионер,0,92342.730612,автомобили
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


Кроме того, если мы посмотрим, какие значения возрастов присутствуют в таблице, то увидим еще один интересный момент:

In [6]:
bank['dob_years'].value_counts()

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

Присутствуют люди, возраст которых равен 0. Эту и описанные выше проблемы мы будем устранять в пункте "Шаг 2. Предобработка данных"

### Вывод

После ознакомления с таблицей мы увидели несколько проблем с данными - отрицательный и дробный показатель для трудового стажа для не пенсионеров, большое значения стажа для пенсионеров, значение возраста у некоторых клиентов равное 0. С этими проблемами мы будем разбираться на следующем шаге.

### Шаг 2. Предобработка данных

Перед тем, как начать обрабатывать пропуски, хотелось бы для начала привести значение стажа пенсионеров к более правдоподобному виду, взять модуль от отрицательного значения стажа и избавиться от нулей в значении возраста, так как они составляют довольно большую часть от наблюдений - примерно 5%.

1) Взглянем на таблицу со значением стоблца `'income_type' = 'пенсионер'` и убедимся, что все значения трудового стажа - аномально большие.

In [7]:
bank[bank['income_type'] == 'пенсионер']['days_employed'].describe()

count      3443.000000
mean     365003.491245
std       21069.606065
min      328728.720605
25%      346649.346146
50%      365213.306266
75%      383231.396871
max      401755.400475
Name: days_employed, dtype: float64

Похоже, что значения трудового стажа для них указано не в днях, а в часах. Для того, чтобы привести их к дням, поделим столбец `'days_employed'` на 24.

In [8]:
bank.loc[bank['income_type'] == 'пенсионер', 'days_employed'] = bank.loc[bank['income_type'] == 'пенсионер', 'days_employed']/24

In [9]:
bank[bank['income_type'] == 'пенсионер']['days_employed'].describe()

count     3443.000000
mean     15208.478802
std        877.900253
min      13697.030025
25%      14443.722756
50%      15217.221094
75%      15967.974870
max      16739.808353
Name: days_employed, dtype: float64

2)Теперь избавимся от отрицательных значений трудового стажа.

In [10]:
bank['days_employed'] = bank['days_employed'].apply(abs)

In [11]:
bank['days_employed'].describe()

count     19351.000000
mean       4677.933504
std        6502.847296
min          24.141633
25%         927.009265
50%        2194.220567
75%        5537.882441
max      395302.838654
Name: days_employed, dtype: float64

3)Заменим нули по следующему принципу - для не пенсионеров значение `days_employed` поделим на 365 и прибавим целую часть от деления к 18(то есть, сделаем предположение, что человек начал работать в 18). Если же это пенсионер, то значение `days_employed` поделим на 365*24, так как,вероятно,значение трудового стажа там указано в часах.

In [12]:
bank.loc[(bank['dob_years'] ==0) & (bank['income_type'] != 'пенсионер'), 'dob_years'] = 18 + abs(bank.loc[bank['dob_years'] ==0, 'days_employed']//365)
bank.loc[(bank['dob_years'] ==0) & (bank['income_type'] == 'пенсионер'), 'dob_years'] = 18 + abs(bank.loc[bank['dob_years'] ==0, 'days_employed']//(365*24))

Убедимся, что нулей в столбце `dob_years` не осталось.

In [13]:
bank[bank['dob_years'] ==0]['dob_years'].count()

0

### Обработка пропусков

Пропущенные значения были только в трудовом стаже и доходе. Сделаем предположение, что все, кто оказались в этой таблице - имеют ненулевой трудовой стаж и общий доход. Тогда можно заменить заменить значения NaN на медианный доход и медианный трудовой стаж по возрастной группе. Такой способ я выбрал потому что считаю, что количество наблюдений для каждого возраста достаточно для того, чтобы по медиане каждой выборке можно было получить близкое к истине пропущенное значение (минимальная выборка - 7 клиентов 73 возраста выглядит достаточной, следовательно, остальные тоже)

In [14]:
bank[pd.isna(bank['total_income']) != True]['dob_years'].value_counts()

35.0    553
41.0    548
38.0    544
40.0    544
34.0    535
42.0    532
33.0    530
39.0    522
44.0    503
31.0    500
29.0    498
36.0    493
48.0    492
37.0    484
30.0    482
32.0    473
50.0    463
43.0    463
27.0    459
49.0    458
45.0    447
28.0    446
56.0    433
52.0    431
46.0    427
54.0    424
47.0    423
53.0    415
59.0    410
58.0    405
57.0    404
51.0    398
55.0    395
26.0    375
60.0    338
25.0    336
61.0    317
62.0    314
24.0    247
63.0    240
64.0    228
23.0    223
22.0    178
65.0    174
66.0    163
67.0    151
21.0    100
68.0     90
69.0     80
70.0     62
20.0     60
71.0     53
19.0     35
72.0     31
18.0      8
73.0      7
74.0      6
75.0      1
Name: dob_years, dtype: int64

In [15]:
bank[pd.isna(bank['total_income']) == True]['dob_years'].value_counts()

34.0    69
40.0    66
42.0    65
31.0    65
35.0    64
36.0    63
47.0    59
41.0    59
30.0    58
28.0    57
57.0    56
58.0    56
54.0    55
38.0    54
56.0    54
52.0    53
37.0    53
39.0    51
33.0    51
50.0    51
45.0    50
51.0    50
29.0    50
49.0    50
43.0    50
55.0    48
46.0    48
48.0    46
44.0    44
53.0    44
60.0    39
62.0    38
61.0    38
64.0    37
32.0    37
27.0    36
23.0    36
26.0    35
59.0    34
63.0    29
25.0    23
24.0    21
65.0    20
66.0    20
21.0    18
22.0    17
67.0    16
68.0     9
20.0     5
69.0     5
71.0     5
70.0     3
72.0     2
19.0     1
73.0     1
Name: dob_years, dtype: int64

Для начала найдем медианные значения стажа и дохода для каждого возраста.

In [16]:
grouped_bank_days = bank.groupby('dob_years')['days_employed'].median()
grouped_bank_income = bank.groupby('dob_years')['total_income'].median()

Теперь заменим все пропущенные значения на медианные(вполне вероятно, что это можно сделать через `apply`, но я потратил кучу времени и не понял как).

In [17]:
for i in range(19,75):
    bank.loc[(bank['dob_years'] == i) & (pd.isna(bank['days_employed']) == True), 'days_employed'] = grouped_bank_days[i]
    bank.loc[(bank['dob_years'] == i) & (pd.isna(bank['total_income']) == True), 'total_income'] = grouped_bank_income[i]

Взглянем теперь на таблицу, остались ли еще там пропущенные значения.

In [18]:
bank[(pd.isna(bank['days_employed']) == True) & (pd.isna(bank['total_income']) == True)]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
1890,0,NaN,NaN,высшее,0,Не женат / не замужем,4,F,сотрудник,0,NaN,жилье
2284,0,NaN,NaN,среднее,1,вдовец / вдова,2,F,пенсионер,0,NaN,недвижимость
4064,1,NaN,NaN,среднее,1,гражданский брак,1,M,компаньон,0,NaN,ремонт жилью
5014,0,NaN,NaN,среднее,1,женат / замужем,0,F,компаньон,0,NaN,покупка недвижимости
6411,0,NaN,NaN,высшее,0,гражданский брак,1,F,пенсионер,0,NaN,свадьба
6670,0,NaN,NaN,Высшее,0,в разводе,3,F,пенсионер,0,NaN,покупка жилой недвижимости
8574,0,NaN,NaN,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,недвижимость
12403,3,NaN,NaN,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,операции с коммерческой недвижимостью
13741,0,NaN,NaN,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,на проведение свадьбы
19829,0,NaN,NaN,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,жилье


Мы получили 10 наблюдений, в которых 3 ячейки с пропущенными значениями и с маленьким потенциалом на восстановление. Их из таблицы мы удалим.

In [19]:
bank = bank.dropna()
bank.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21515 entries, 0 to 21524
Data columns (total 12 columns):
children            21515 non-null int64
days_employed       21515 non-null float64
dob_years           21515 non-null float64
education           21515 non-null object
education_id        21515 non-null int64
family_status       21515 non-null object
family_status_id    21515 non-null int64
gender              21515 non-null object
income_type         21515 non-null object
debt                21515 non-null int64
total_income        21515 non-null float64
purpose             21515 non-null object
dtypes: float64(3), int64(4), object(5)
memory usage: 2.1+ MB


Пропусков в наших данных больше нет - можем двигаться дальше!

### Вывод

В шаге 2 мы исправили множество проблем, которые имели данные датасета, а именно - заполнили все NaN значения, сделали положительными отрицательные значения трудового стажа и избавились от значения 0 в столбце `dob_years`.

### Замена типа данных

Можно обозначить два столбца, которым можно заменить тип данных. `dob_years` и `days_employed` было бы логичней представить в виде типа данных integer. Для этого используем метод `astype()`.

In [20]:
bank['days_employed'] = bank['days_employed'].astype('int')
bank['dob_years'] = bank['dob_years'].astype('int')

In [21]:
bank.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21515 entries, 0 to 21524
Data columns (total 12 columns):
children            21515 non-null int64
days_employed       21515 non-null int64
dob_years           21515 non-null int64
education           21515 non-null object
education_id        21515 non-null int64
family_status       21515 non-null object
family_status_id    21515 non-null int64
gender              21515 non-null object
income_type         21515 non-null object
debt                21515 non-null int64
total_income        21515 non-null float64
purpose             21515 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.1+ MB


In [22]:
bank.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


### Вывод

Были обнаружены два стобца, где можно изменить тип данных - это `dob_years` и `days_employed`, что и было сделано.

### Обработка дубликатов

Перед тем, как начать обрабатывать дубликаты, надо просмотреть столбцы на наличие в них аномальных значений.

In [23]:
bank['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

Вероятно, что количество детей = 20 могло появиться от неверной интерпретации числа 2, а значение -1, от неверной интерпретации числа 1. Основываясь на этой утверждении, заменим 20 на 2 и -1 на 1.

In [24]:
bank.loc[bank['children'] == -1, 'children'] = 1
bank.loc[bank['children'] == 20, 'children'] = 2

In [25]:
bank['children'].unique()

array([1, 0, 3, 2, 4, 5])

In [26]:
bank['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

Видим, что есть много одинаковых значений образования, но написанных в разных регистрах, что выдаст нам в итоге разный результат при группировке. Нужно привести их к одному регистру. Сделаем это с помощью метода `str.lower()`.

In [27]:
bank['education'] = bank['education'].str.lower()

In [28]:
bank['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

In [29]:
bank['education_id'].unique()

array([0, 1, 2, 3, 4])

In [30]:
bank['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [31]:
bank['family_status_id'].unique()

array([0, 1, 2, 3, 4])

In [32]:
bank['gender'].unique()

array(['F', 'M', 'XNA'], dtype=object)

In [33]:
bank['income_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

In [34]:
bank['debt'].unique()

array([0, 1])

In [35]:
bank['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Во всех столбцах устранены аномальные значения, теперь можно приступить к поиску дубликатов с помощью метода `duplicated()`.

In [36]:
bank.duplicated().sum()

71

Мы видим, что дубликаты есть. Появиться они могли из - за технических проблем или невнимательности человека, который занес данные два раза подряд.

Удалим дубликаты с помощью метода `drop_duplicates()`.

In [37]:
bank = bank.drop_duplicates()

In [38]:
bank.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21444 entries, 0 to 21524
Data columns (total 12 columns):
children            21444 non-null int64
days_employed       21444 non-null int64
dob_years           21444 non-null int64
education           21444 non-null object
education_id        21444 non-null int64
family_status       21444 non-null object
family_status_id    21444 non-null int64
gender              21444 non-null object
income_type         21444 non-null object
debt                21444 non-null int64
total_income        21444 non-null float64
purpose             21444 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.1+ MB


### Вывод

Мы разобрались с аномальными значениями в различных столбцах - где - то нужно было заменить отрицательные и слишком большие значения, а где то значения были одинаковыми, но написанными в разном регистре, что затруднило бы анализ данных в дальнейшем. Сейчас такой проблемы больше нет.

### Лемматизация

Для того, чтобы провести лемматизацию, сначала изучим, какие значения присутствуют в столбце `purpose`.

In [39]:
purposes = bank['purpose'].unique()
purposes

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Внимательно изучив все причины, можно выделить 4 общих цели взятия кредита: операции с *недвижимостью(жильем), автомобилем, свадьбой и образованием*. Для того, чтобы сгруппировать эти значения, проведем лемматизацию столбца с помощью функции `Mystem.lemmatize бибилиотеки  pymystem3`.

In [40]:
m = Mystem()
def lemmma_purposes(purpose):
    lemma = m.lemmatize(purpose)
    if ('жилье' in lemma) or ('недвижимость' in lemma):
        return 'недвижимость'
    elif 'автомобиль' in lemma:
        return 'автомобиль'
    elif 'образование' in lemma:
        return 'образование'
    elif 'свадьба' in lemma:
        return 'свадьба'


In [41]:
bank['purpose'] = bank['purpose'].apply(lemmma_purposes)

In [42]:
bank['purpose'].unique()

array(['недвижимость', 'автомобиль', 'образование', 'свадьба'],
      dtype=object)

### Вывод

В данном пункте была проведена лемматизация целей кредита. Было выявлено 4 основных цели, зачем люди берут кредиты - недвижимость(жилье), автомобиль, свадьба и образование.

### Категоризация данных

Попробуем выделить категории на основании возраста - *молодой(до 35 лет), средний возраст(35 - 55), пенсионер(56 и больше)* и семейного положения и количества детей - *одинокий(количество детей - 0 и family_status - Не женат / не замужем, вдовец / вдова, в разводе) или не одинокий(количество детей > 0 или family_status - женат / замужем, гражданский брак)*
Я выделил такие категории, потому что в категориях, связаных с возрастом мы выделяем людей разных поколений, разного мышления, что может повлиять на процент возврата кредита. Что касается семейного статуса, то идея проста - если человек одинокий, то все свои деньги он тратит на себя и, возможно, вернуть кредит ему будет легче, чем когда у него есть семья.

Напишем 2 функции: функцию для определения категории возраста по количеству лет и функцию, которая по количеству детей и семейному статусу, говорит, одинок клиент или нет.

In [43]:
def age_status(age):
    if age <= 35:
        return 'молодой'
    elif 35 < age < 60:
        return 'средний возраст'
    else: return 'пожилой'

In [44]:
def alone_or_not(data):
    family_status = data['family_status']
    children = data['children']
    alone = ["Не женат / не замужем", "вдовец / вдова", "в разводе"]
    
    if children > 0:
        return 'не одинокий'
    elif family_status in alone:
        return 'одинокий'
    else: return 'не одинокий'
    

In [45]:
bank['age_status'] = bank['dob_years'].apply(age_status)

In [46]:
bank['alone_status'] = bank.apply(alone_or_not, axis=1)

In [47]:
bank.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_status,alone_status
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,недвижимость,средний возраст,не одинокий
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,автомобиль,средний возраст,не одинокий
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,недвижимость,молодой,не одинокий
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,образование,молодой,не одинокий
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,свадьба,средний возраст,не одинокий


### Вывод

На основании имеющихся данных удалось выделить 2 категории: первое разделение подразумевает выделение возрастных категорий по возрасту, а второе определяет, одинок человек или нет по количеству детей и семейному положению.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Для того, чтобы ответить на этот вопрос, разделим таблицу на 2 части по параметру `bank['children'] >0` и `bank['children'] == 0` и посмотрим на процент отданных кредитов в срок.

In [48]:
bank_no_children = bank[bank['children'] == 0]
grouped_no_children = bank_no_children.groupby('debt')['debt'].count()
percent_no = grouped_no_children[1]/bank_no_children.shape[0]
print('Процент кредитов, не возвращенных в срок, среди семей без детей - {:.2%}'.format(percent_no))

Процент кредитов, не возвращенных в срок, среди семей без детей - 7.55%


In [49]:
bank_yes_children = bank[bank['children'] > 0]
grouped_yes_children = bank_yes_children.groupby('debt')['debt'].count()
percent_yes = grouped_yes_children[1]/bank_yes_children.shape[0]
print('Процент кредитов, не возвращенных в срок, среди семей с детьми - {:.2%}'.format(percent_yes))

Процент кредитов, не возвращенных в срок, среди семей с детьми - 9.21%


На мой взгляд, результаты получились довольно похожими, поэтому нельзя говорить о том, что есть зависимость между наличием детей и возвратом кредита в срок.

### Вывод

Зависимости нет.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

Составим сводную таблицу, чтобы увидеть распределение возврата и невозврата кредита для каждого типа семейного положения.

In [50]:
bank_pivot_family = bank.pivot_table(index= 'family_status', columns = 'debt', values = 'purpose',  aggfunc = 'count')
bank_pivot_family_res = bank_pivot_family.reset_index()

debt,0,1
family_status,,
Не женат / не замужем,2535,274
в разводе,1109,85
вдовец / вдова,895,63
гражданский брак,3760,388
женат / замужем,11404,931


Посчитаем отношение невозвращенных кредитов к общему числу кредитов для каждой категории.

In [52]:
bank_pivot_family_res['proportion'] = (bank_pivot_family_res[1]/(bank_pivot_family_res[0] + bank_pivot_family_res[1]))

In [53]:
bank_pivot_family_res

debt,family_status,0,1,proportion
0,Не женат / не замужем,2535,274,0.097544
1,в разводе,1109,85,0.071189
2,вдовец / вдова,895,63,0.065762
3,гражданский брак,3760,388,0.093539
4,женат / замужем,11404,931,0.075476


In [54]:
print('Процент кредитов, не возвращенных в срок, среди клиентов с семейным положением "Не женат / не замужем" - {:.2%}'.format(bank_pivot_family_res['proportion'][0]))
print('Процент кредитов, не возвращенных в срок, среди клиентов с семейным положением "в разводе" - {:.2%}'.format(bank_pivot_family_res['proportion'][1]))
print('Процент кредитов, не возвращенных в срок, среди клиентов с семейным положением "вдовец / вдова" - {:.2%}'.format(bank_pivot_family_res['proportion'][2]))
print('Процент кредитов, не возвращенных в срок, среди клиентов с семейным положением "гражданский брак" - {:.2%}'.format(bank_pivot_family_res['proportion'][3]))
print('Процент кредитов, не возвращенных в срок, среди клиентов с семейным положением "женат / замужем" - {:.2%}'.format(bank_pivot_family_res['proportion'][4]))

Процент кредитов, не возвращенных в срок, среди клиентов с семейным положением "Не женат / не замужем" - 9.75%
Процент кредитов, не возвращенных в срок, среди клиентов с семейным положением "в разводе" - 7.12%
Процент кредитов, не возвращенных в срок, среди клиентов с семейным положением "вдовец / вдова" - 6.58%
Процент кредитов, не возвращенных в срок, среди клиентов с семейным положением "гражданский брак" - 9.35%
Процент кредитов, не возвращенных в срок, среди клиентов с семейным положением "женат / замужем" - 7.55%


Самый большой процент по невозврату долгов можно увидеть у клиентов с семейным положением "Не женат / не замужем" - 9.75%, самый маленький у "вдовец / вдова" - 6.58%. На мой взгляд, на основании полученных результатов, нельзя сказать, что существует зависимость между семейным положением и возвратом кредита в срок.

### Вывод

Зависимости нет.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Разделим весь доход на маленький (последние 25%), средний(от 25 до 75%) и высокий(выше 75%)

In [55]:
bank['total_income'].describe()['25%']

107499.81581452076

In [56]:
bank['total_income'].describe()['75%']

195842.40622523567

Напишем функцию, которая будет брать информацию из ячейки `data['total_income']` и на основании данных в ней выдавать категорию дохода клиента.

In [57]:
def income_level(data):
    # в переменные income_25 и income_75 помещаем значения 25 и 75 процентных квантилей столбца 'total_income' соответственно. 
    # Их мы получили с помощью метода describe выше.
    income_25 = 107499.81581452076
    income_75 = 195842.40622523567
    income = data['total_income']
    
    if income < income_25:
        return 'маленький'
    elif income > income_75:
        return 'большой'
    else: return 'средний'

In [58]:
bank['income_level'] = bank.apply(income_level, axis = 1)

In [59]:
bank.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_status,alone_status,income_level
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,недвижимость,средний возраст,не одинокий,большой
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,автомобиль,средний возраст,не одинокий,средний
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,недвижимость,молодой,не одинокий,средний
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,образование,молодой,не одинокий,большой
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,свадьба,средний возраст,не одинокий,средний


Теперь сгруппируем данные по уровню дохода и посмотрим на процент кредитов, не отданных в срок.

In [70]:
bank_low_income = bank[bank['income_level'] == 'маленький']
grouped_low_income = bank_low_income.groupby('debt')['debt'].count()
percent_low_income = grouped_low_income[1]/bank_low_income.shape[0]
print('Процент кредитов, не возвращенных в срок, среди клиентов с маленьким достатком - {:.2%}'.format(percent_low_income))

Процент кредитов, не возвращенных в срок, среди клиентов с маленьким достатком - 7.96%


In [61]:
bank_middle_income = bank[bank['income_level'] == 'средний']
grouped_middle_income = bank_middle_income.groupby('debt')['debt'].count()
percent_middle_income = grouped_middle_income[1]/bank_middle_income.shape[0]
print('Процент кредитов, не возвращенных в срок, среди клиентов со средним достатком - {:.2%}'.format(percent_middle_income))

Процент кредитов, не возвращенных в срок, среди клиентов со средним достатком - 8.68%


In [62]:
bank_big_income = bank[bank['income_level'] == 'большой']
grouped_big_income = bank_big_income.groupby('debt')['debt'].count()
percent_big_income = grouped_big_income[1]/bank_big_income.shape[0]
print('Процент кредитов, не возвращенных в срок, среди клиентов с большим достатком - {:.2%}'.format(percent_big_income))

Процент кредитов, не возвращенных в срок, среди клиентов с большим достатком - 7.14%


Хуже всего возвращают кредит люди со средним достатком - 8.68% невозвращенных кредитов. Меньше просрочек у людей с высоким достатком - 7.14%. Но говорить о качественных различиях не приходится, поэтому на основании полученных результатов нельзя сделать вывод, что есть зависимость между уровнем дохода и возвратом кредита в срок.

### Вывод

Зависимости нет.

- Как разные цели кредита влияют на его возврат в срок?

Составим сводную таблицу, чтобы увидеть распределение возврата и невозврата кредита для каждой цели взятия кредита.

In [65]:
bank_pivot_purpose = bank.pivot_table(index= 'purpose', columns = 'debt', values = 'education_id',  aggfunc = 'count')
bank_pivot_purpose_res = bank_pivot_purpose.reset_index()

debt,0,1
purpose,,
автомобиль,3903,403
недвижимость,10021,782
образование,3643,370
свадьба,2136,186


Посчитаем отношение невозвращенных кредитов к общему числу кредитов для каждой категории.

In [67]:
bank_pivot_purpose_res['proportion'] = (bank_pivot_purpose_res[1]/(bank_pivot_purpose_res[0] + bank_pivot_purpose_res[1]))

In [68]:
bank_pivot_purpose_res

debt,purpose,0,1,proportion
0,автомобиль,3903,403,0.093590
1,недвижимость,10021,782,0.072387
2,образование,3643,370,0.092200
3,свадьба,2136,186,0.080103


In [69]:
print('Процент кредитов, не возвращенных в срок, имевших цель "автомобиль" - {:.2%}'.format(bank_pivot_family_res['proportion'][0]))
print('Процент кредитов, не возвращенных в срок, имевших цель "недвижимость" - {:.2%}'.format(bank_pivot_family_res['proportion'][1]))
print('Процент кредитов, не возвращенных в срок, имевших цель "образование" - {:.2%}'.format(bank_pivot_family_res['proportion'][2]))
print('Процент кредитов, не возвращенных в срок, имевших цель "свадьба" - {:.2%}'.format(bank_pivot_family_res['proportion'][3]))

Процент кредитов, не возвращенных в срок, имевших цель "автомобиль" - 9.75%
Процент кредитов, не возвращенных в срок, имевших цель "недвижимость" - 7.12%
Процент кредитов, не возвращенных в срок, имевших цель "образование" - 6.58%
Процент кредитов, не возвращенных в срок, имевших цель "свадьба" - 9.35%


По получившимся данным можно сделать вывод, что чаще кредит возвращают вовремя те, кто брал его на образование, а больше задержек у тех, кто брал деньги на автомобиль. Но говорить о том, что различия этих результат достаточно большие, я не могу. Возможно, это лишь частный случай. Поэтому, я бы сказал, что основываясь на наших данных, нельзя сделать вывод о том, что цель кредита оказывает влияние на возврат кредита в срок.

### Вывод

Цели кредита не оказывают влияние на возврат кредита в срок.

### Шаг 4. Общий вывод

Был исследован датафрейм с информацией по клиентам банка, берущим в нем кредит. В ходе исследования пришлось столкнуться со множеством проблем, так как данные оказались 'сырыми'. Были исправлены такие недочеты как пустые значения, отрицательные значения, неверный тип данных, аномальные значения, дубликаты. Кроме того, была проведена лемматизация целей получения кредита, что позволило выделить основные цели получения кредита.
Был проведен анализ зависимости выплаты кредита в срок от разных категорий данных. К сожалению, не удалось выявить зависимость между какой - либо категорией данных и выплатой кредита в срок.